This notebook contains code to generate feedback from the Collie dataset. It depends on having the Collie repo (https://github.com/princeton-nlp/Collie) cloned in the same directory as this notebook. 

This notebook takes five of the datasets and correspodning tasks from Collie, mainly:
wiki_co7/guten_c07 - every sentence contain words (actual: full response contains words)
ccnews_c08 - all sentences have 1st word (actual: response starts with given 1st word)
wiki_c09 - sentence count, no words (actual: only no words words)

However, because it is difficult to distinguish what is a sentence vs an abbreviation with a period and space, we exclude the sentence constraints or modify them

In [1]:
import dill
from pathlib import Path
import sys

In [2]:
sys.path.append("Collie")
with open("Collie/data/all_data.dill", "rb") as f:
    all_data = dill.load(f)

In [19]:
out = open("../feedback/collie.py", "w+")
out.write("from src.dataset.feedback_utils import Feedback, Scope, Type, Metric, Comparison\n\ncollie_feedback = [\n")

102

In [13]:
def get_feedback(prompt, domain, categories, metric, metric_value, comparison="Comparison.greater_than"):
    return """
    Feedback(
        content="{prompt}",
        domain="{domain}",
        effect="placeholder to fix ValidationError",
        scope=Scope.regional,
        categories={categories},
        type=Type.quantitative,
        metric={metric},
        metric_value={metric_value},
        comparison={comparison}
    ),
""".format(prompt=prompt, domain=domain, categories=categories, metric=metric, metric_value=metric_value, comparison=comparison)

In [20]:
import numpy as np
key = "wiki_c07"
chosen = np.random.choice(np.arange(len(all_data[key])), size=38, replace=False)
for i in chosen:
    obj = all_data[key][i]
    topic = obj['metadata']["title"]
    words = obj['prompt'].split("containing the word")[1][:-1].strip()
    prompt = f"When talking about {topic}, make sure all sentences contain the words {words}"
    domain = f"Talking about {topic}"
    out_str = get_feedback(prompt, domain, f"['collie', '{key}']", "Metric.contains_all_strings", """[{words}]""".format(words=words))
    out.write(out_str + "\n")

In [21]:
key = "ccnews_c08/wiki_c08"
arr = []
arr.extend(all_data["wiki_c08"])
arr.extend(all_data["ccnews_c08"])
chosen = np.random.choice(np.arange(len(arr)), size=24, replace=False)
print(len(arr))
for i in chosen:
    obj = arr[i]
    topic = obj['metadata']["title"]
    words = obj['prompt'].split("1st word to be")[1][:-1].strip()
    prompt = f"When talking about {topic}, make sure the first sentences has a 1st word of {words}"
    domain = f"Talking about {topic}"
    out_str = get_feedback(prompt, domain, f"['collie', '{key}']", "Metric.first_words",words.lower())
    out.write(out_str + "\n")

25


In [17]:
key = "wiki_c09"
chosen = np.random.choice(np.arange(len(all_data[key])), size=38, replace=False)
for i in chosen:
    obj = all_data[key][i]
    topic = obj['metadata']["title"]
    targets = obj["targets"]
    words = "\'" + "', '".join(targets[1:]) + "'"
    prompt = f"When talking about {topic}, do not use the words {words}"
    domain = f"Talking about {topic}"
    out_str = get_feedback(prompt, domain, f"['collie', '{key}']", "Metric.contains_none_strings", f"{targets[1:]}")
    out.write(out_str + "\n")

In [18]:
out.write("]\n")
out.close()